In [1]:
import json
import pandas as pd
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import numpy as np
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from datetime import datetime, date


In [2]:
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/negar.json"



Mounted at /content/drive


In [ ]:
# Load Telegram JSON file
with open( file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract messages
messages = data["messages"]
friend_name = data["name"]

# Convert to DataFrame
df = pd.DataFrame([
    {
        "date": msg["date"][:10],  # Keep only YYYY-MM-DD
        "from": msg.get("from", "Unknown"),  # Use 'Unknown' if sender info is missing
        "text": msg["text"] if isinstance(msg["text"], str) else "",  # Keep only text messages
        "friend_name" : friend_name
    }
    for msg in messages if "date" in msg and "text" in msg
])

df

,date,from,text,friend_name
0,2025-02-02,Negar♥️,رو صورتم,Negar♥️
1,2025-02-02,Negar♥️,از این جوشای خاااااااااااااار زده,Negar♥️
2,2025-02-02,Negar♥️,بالای لبم,Negar♥️
3,2025-02-02,Negar♥️,رو پیشونیم,Negar♥️
4,2025-02-02,Negar♥️,رو لپم,Negar♥️
...,...,...,...,...
828,2025-02-20,Niloofar Torki,باشه حالا صبر کن,Negar♥️
829,2025-02-20,Niloofar Torki,همه در اومدن,Negar♥️
830,2025-02-20,Niloofar Torki,تو هم در بیا,Negar♥️
831,2025-02-20,Negar♥️,نل پیام نده اینجا فعلا,Negar♥️


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("persiannlp/mt5-large-parsinlu-sentiment-analysis")
model = AutoModelForSeq2SeqLM.from_pretrained("persiannlp/mt5-large-parsinlu-sentiment-analysis")
from transformers import AutoModelForSequenceClassification, AutoConfig



def run_model(messages, **generator_args):
    # Join messages safely
    context = " <sep> ".join(messages)

    # Tokenize and generate output
    input_ids = tokenizer.encode(context, return_tensors="pt")
    res = model.generate(input_ids)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    print(output)

    return output[0]


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
def process_messages(messages):
    joined_text = " <sep> ".join(messages)  # Join messages with "<sep>"
    return joined_text  # Example processing (replace with model function)

# Apply the function to each date
df_grouped = df.groupby('date')['text'].apply(list).reset_index()  # Get list of messages per date  # Apply function

# Print results
df_grouped

,date,text
0,2025-02-02,"[رو صورتم, از این جوشای خاااااااااااااار زده, ..."
1,2025-02-06,"[عبدلی آهنگ گذاشته, کلا دوتا اهنگ, یکی به من ن..."
2,2025-02-07,"[, داشتم تو اینستا گوش مدادم همینو, 😭😭😭, , , ا..."
3,2025-02-08,"[, ویدیوی پر شده در ایران, اااااااا اهنگ موردع..."
4,2025-02-09,[کاشکی این اهنگ ای دللللللللل دل دیوونه پاک مش...
5,2025-02-10,[]
6,2025-02-11,"[زودپز هشت و نینه اصلا, , عامو زودپز میخواهیم ..."
7,2025-02-13,"[, این من تو شرکت زدم پرینتر رنگاش خوب نبود, ,..."
8,2025-02-14,"[چقهههههه ادم پول دوستیه, و اینکه قرار شد بابت..."
9,2025-02-15,"[دلم صدا مدههههههههه, دیشب مسخره مامان کردیم, ..."


In [ ]:
# df_grouped["sentiment"] = df_grouped["text"].apply(run_model)

In [ ]:
sentiment_mapping = {
    'very negative': -1,
    'negative': -0.5,
    'neutral': 0,
    'no sentiment expressed' : 0,
    'mixed' : 0,
    'positive': 0.5,
    'very positive': 1
}


sentiment_result = df.groupby("date").agg(
    messages = ("text", list),
    no_of_messages = ("text", "count")
).reset_index()

sentiment_result["sentiment_result"] = sentiment_result["messages"].apply(run_model)


sentiment_result["sentiment_score"] = sentiment_result["sentiment_result"].map(sentiment_mapping)

sentiment_result["sentiment_score"] = sentiment_result["sentiment_score"] * sentiment_result["no_of_messages"]

print(sentiment_result)

total_sentiment_score = sentiment_result["sentiment_score"].sum()/ sentiment_result["no_of_messages"].sum()
total_no_of_messages_analyzed = sentiment_result["no_of_messages"].sum()

# sentiment_results
print(total_sentiment_score)


# Filter messages from Niloofar Torki
df_filtered = df[df["from"] == "Niloofar Torki"]

your_sentiment_result = df_filtered.groupby("date").agg(
    messages = ("text", list),
    no_of_messages = ("text", "count")
).reset_index()

# Group by 'date' and apply sentiment analysis (run_model)
your_sentiment_result["sentiment_result"] = your_sentiment_result["messages"].apply(run_model)


your_sentiment_result["sentiment_score"] = your_sentiment_result["sentiment_result"].map(sentiment_mapping)
your_sentiment_result["sentiment_score"] = your_sentiment_result["sentiment_score"] * your_sentiment_result["no_of_messages"]
total_no_of_your_messages = your_sentiment_result["no_of_messages"].sum()
your_sentiment_score = your_sentiment_result["sentiment_score"].sum()/total_no_of_your_messages
print(your_sentiment_result)
print(your_sentiment_score, total_no_of_your_messages)
# print(your_sentiment_score)

# Filter messages from Niloofar Torki
df_filtered = df[df['from'].str.contains(friend_name, na=False)]

friend_sentiment_result = df_filtered.groupby("date").agg(
    messages = ("text", list),
    no_of_messages = ("text", "count")
).reset_index()

# Group by 'date' and apply sentiment analysis (run_model)
friend_sentiment_result["sentiment_result"] =friend_sentiment_result["messages"].apply(run_model)

friend_sentiment_result["sentiment_score"] = friend_sentiment_result["sentiment_result"].map(sentiment_mapping)
friend_sentiment_result["sentiment_score"] = friend_sentiment_result["sentiment_score"] * friend_sentiment_result["no_of_messages"]
friend_total_no_of_messages = friend_sentiment_result["no_of_messages"].sum()
friend_sentiment_score = friend_sentiment_result["sentiment_score"].sum()/friend_total_no_of_messages

print(friend_sentiment_result)
print(friend_sentiment_score, friend_total_no_of_messages)
# print(friend_sentiment_score)

# df


['very negative']
['very negative']
['very negative']
['no sentiment expressed']
['very negative']
['negative']
['very negative']
['negative']
['no sentiment expressed']
['negative']
['no sentiment expressed']
['very negative']
['very negative']
          date                                           messages  \
0   2025-02-02  [رو صورتم, از این جوشای خاااااااااااااار زده, ...   
1   2025-02-06  [عبدلی آهنگ گذاشته, کلا دوتا اهنگ, یکی به من ن...   
2   2025-02-07  [, داشتم تو اینستا گوش مدادم همینو, 😭😭😭, , , ا...   
3   2025-02-08  [, ویدیوی پر شده در ایران, اااااااا اهنگ موردع...   
4   2025-02-09  [کاشکی این اهنگ ای دللللللللل دل دیوونه پاک مش...   
5   2025-02-10                                                 []   
6   2025-02-11  [زودپز هشت و نینه اصلا, , عامو زودپز میخواهیم ...   
7   2025-02-13  [, این من تو شرکت زدم پرینتر رنگاش خوب نبود, ,...   
8   2025-02-14  [چقهههههه ادم پول دوستیه, و اینکه قرار شد بابت...   
9   2025-02-15  [دلم صدا مدههههههههه, دیشب مسخره مامان کردیم, ..

In [ ]:
print(total_no_of_messages_analyzed)

108


In [19]:
suggestion_df = pd.read_json('/content/drive/My Drive/suggestion.json', orient='records', lines=True)
suggestion_df["latest_chat"] = pd.to_datetime(suggestion_df["latest_chat"], unit="ms")
suggestion_df.drop(columns= ["AI_suggestion"], inplace = True)
suggestion_df




,username,Friend_name,total_sentiment_score,total_messages,your_sentiment_score,your_messages,friend_sentiment_score,friend_messages,latest_chat,dates,average time duration(day),suggestion
0,Niloofar Torki,Delara,0.841712,100,0.596154,50,0.729167,50,1970-01-01 00:29:00.096,,1,"You should text Delara, it's been a long time."
1,Niloofar Torki,Mahshid👭🏻♥️,-0.243598,1640,-0.101508,862,-0.771851,778,1970-01-01 00:29:00.268,"[2025-01-02, 2025-01-03, 2025-01-04, 2025-01-0...",1,"You should text Mahshid👭🏻♥️, it's been a long ..."
2,Niloofar Torki,Negar♥️,-0.701681,833,-0.333910,289,-0.756481,540,2025-02-20 00:00:00.000,"[2025-02-02, 2025-02-06, 2025-02-07, 2025-02-0...",1,"You should text Negar♥️, it's been a long time."


In [17]:
suggestion_df

,username,Friend_name,total_sentiment_score,total_messages,your_sentiment_score,your_messages,friend_sentiment_score,friend_messages,latest_chat,dates,average time duration(day),suggestion,AI_suggestion
0,Niloofar Torki,Delara,0.841712,100,0.596154,50,0.729167,50,1970-01-01 00:29:00.096,,1,"You should text Delara, it's been a long time.","As a friendship advisor, evaluate my friendshi..."
1,Niloofar Torki,Mahshid👭🏻♥️,-0.243598,1640,-0.101508,862,-0.771851,778,1970-01-01 00:29:00.268,"[2025-01-02, 2025-01-03, 2025-01-04, 2025-01-0...",1,"You should text Mahshid👭🏻♥️, it's been a long ...","As a friendship advisor, evaluate my friendshi..."
2,Niloofar Torki,Negar♥️,-0.701681,833,-0.333910,289,-0.756481,540,2025-02-20 00:00:00.000,"[2025-02-02, 2025-02-06, 2025-02-07, 2025-02-0...",1,"You should text Negar♥️, it's been a long time.","As a friendship advisor, evaluate my friendshi..."


In [ ]:
def update_df(row):
  if row["Friend_name"] == friend_name:
      dates = df["date"].unique().tolist()
      print(row["dates"])
      previous_date = row["dates"]
      for i in dates:
        if i not in previous_date:
          previous_date.append(i)
      new_dates = previous_date
      dates_df = pd.to_datetime(new_dates)
      dates_diff = dates_df.diff().mean()

      last_analysis_total = row["total_sentiment_score"] * row["total_messages"]
      print(last_analysis_total)
      last_analysis_you = row["your_sentiment_score"] * row["your_messages"]
      print(last_analysis_you)
      last_analysis_friend = row["friend_sentiment_score"] * row["friend_messages"]
      print(last_analysis_friend)

      new_count_total = (total_no_of_messages_analyzed + row["total_messages"])
      new_count_you = (total_no_of_your_messages + row["your_messages"])
      new_count_friend = (friend_total_no_of_messages + row["friend_messages"])
      print(new_count_total, new_count_you, new_count_friend)

      new_total_score = (total_sentiment_score * total_no_of_messages_analyzed + last_analysis_total)/ new_count_total
      new_your_score = (your_sentiment_score * total_no_of_your_messages  + last_analysis_you)/ new_count_you
      new_friend_score = (friend_sentiment_score * friend_total_no_of_messages + last_analysis_friend) / new_count_friend

      print(new_total_score, new_your_score, new_friend_score)


      new_dates_diff = dates_diff.days

      new_date = df["date"].iloc[-1]

      row["total_sentiment_score"] = new_total_score
      row["total_messages"] = new_count_total
      row["your_sentiment_score"] = new_your_score
      row["your_messages"] = new_count_you
      row["friend_sentiment_score"] = new_friend_score
      row["friend_messages"] = new_count_friend
      row["latest_chat"] = new_date
      row["dates"] = list(new_dates)
      row["average time duration(day)"] = new_dates_diff
      row["suggestion"] = ""

      return row

  else:
    return row


suggestion_df = suggestion_df.apply(update_df, axis = 1)


if friend_name not in list(suggestion_df["Friend_name"]):
  dates = df["date"].unique().tolist()
  print(type(dates))
  dates_df = pd.to_datetime(dates)
  dates_diff = dates_df.diff().mean()

  suggestion_df.loc[len(suggestion_df)] = ["Niloofar Torki",
                                            friend_name,
                                            total_sentiment_score,
                                            total_no_of_messages_analyzed,
                                            your_sentiment_score,
                                            total_no_of_your_messages,
                                            friend_sentiment_score,
                                            friend_total_no_of_messages,
                                            df["date"].iloc[-1],
                                            list(dates),
                                            dates_diff.days,
                                            ""]

suggestion_df['latest_chat'] = pd.to_datetime(suggestion_df['latest_chat'])

# row
suggestion_df


<class 'list'>


,username,Friend_name,total_sentiment_score,total_messages,your_sentiment_score,your_messages,friend_sentiment_score,friend_messages,latest_chat,dates,average time duration(day),suggestion
0,Niloofar Torki,Delara,0.841712,100,0.596154,50,0.729167,50,1970-01-01 00:29:00.096000,,1,"You should text Delara, it's been a long time."
1,Niloofar Torki,Mahshid👭🏻♥️,-0.243598,1640,-0.101508,862,-0.771851,778,1970-01-01 00:29:00.268800,"[2025-01-02, 2025-01-03, 2025-01-04, 2025-01-0...",1,It's better to pay more attention to Mahshid👭🏻...
2,Niloofar Torki,Negar♥️,-0.701681,833,-0.333910,289,-0.756481,540,2025-02-20 00:00:00.000000,"[2025-02-02, 2025-02-06, 2025-02-07, 2025-02-0...",1,


In [ ]:
def suggest(row):
  today = date.today()
  if row["your_sentiment_score"] <= -0.5:
    return f"It seems that {row['Friend_name']} brings you down. I suggest not communicating with {row['Friend_name']}."

  elif (pd.Timestamp.today() - row["latest_chat"]).days > row["average time duration(day)"]:
   return f"You should text {row['Friend_name']}, it's been a long time."

  elif row["friend_sentiment_score"] < 0:
   return f"It's better to pay more attention to {row['Friend_name']}. She seems sad lately."

suggestion_df["suggestion"] = suggestion_df.apply(suggest, axis= 1)
suggestion_df

,username,Friend_name,total_sentiment_score,total_messages,your_sentiment_score,your_messages,friend_sentiment_score,friend_messages,latest_chat,dates,average time duration(day),suggestion
0,Niloofar Torki,Delara,0.841712,100,0.596154,50,0.729167,50,1970-01-01 00:29:00.096000,,1,"You should text Delara, it's been a long time."
1,Niloofar Torki,Mahshid👭🏻♥️,-0.243598,1640,-0.101508,862,-0.771851,778,1970-01-01 00:29:00.268800,"[2025-01-02, 2025-01-03, 2025-01-04, 2025-01-0...",1,"You should text Mahshid👭🏻♥️, it's been a long ..."
2,Niloofar Torki,Negar♥️,-0.701681,833,-0.333910,289,-0.756481,540,2025-02-20 00:00:00.000000,"[2025-02-02, 2025-02-06, 2025-02-07, 2025-02-0...",1,"You should text Negar♥️, it's been a long time."


In [20]:
suggestion_df.to_json('/content/drive/My Drive/suggestion.json', orient='records', lines=True)